In [ ]:
########################################
# space prep
########################################

In [ ]:
!pip install findspark
!pip install pyspark
!apt-get install -qq openjdk-17-jdk-headless
from google.colab import drive

Selecting previously unselected package openjdk-17-jre-headless:amd64.
(Reading database ... 126101 files and directories currently installed.)
Preparing to unpack .../openjdk-17-jre-headless_17.0.14+7-1~22.04.1_amd64.deb ...
Unpacking openjdk-17-jre-headless:amd64 (17.0.14+7-1~22.04.1) ...
Selecting previously unselected package openjdk-17-jdk-headless:amd64.
Preparing to unpack .../openjdk-17-jdk-headless_17.0.14+7-1~22.04.1_amd64.deb ...
Unpacking openjdk-17-jdk-headless:amd64 (17.0.14+7-1~22.04.1) ...
Setting up openjdk-17-jre-headless:amd64 (17.0.14+7-1~22.04.1) ...
update-alternatives: using /usr/lib/jvm/java-17-openjdk-amd64/bin/java to provide /usr/bin/java (java) in auto mode
update-alternatives: using /usr/lib/jvm/java-17-openjdk-amd64/bin/jpackage to provide /usr/bin/jpackage (jpackage) in auto mode
update-alternatives: using /usr/lib/jvm/java-17-openjdk-amd64/bin/keytool to provide /usr/bin/keytool (keytool) in auto mode
update-alternatives: using /usr/lib/jvm/java-17-openj

In [ ]:
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
#path_base = "/content/drive/MyDrive/dataset/CarAccidents/"
path_base = "/content/drive/MyDrive/CarAccidents/"
import findspark, os
findspark.init()

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("CarAccidents")
    .master("local[*]") #na vsetkych jadrach
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")
#accidents_df  = spark.read.option("header", True).csv("/content/drive/MyDrive/dataset/CarAccidents/Accidents.csv")
accidents_df  = spark.read.option("header", True).csv(path_base + "Accidents.csv")
vehicles_df   = spark.read.option("header", True).csv(path_base + "Vehicles.csv")
casualties_df = spark.read.option("header", True).csv(path_base + "Casualties.csv")

In [ ]:
##########################################################
# Integrácia dát -  integrácia všetkých troch tabuliek.
##########################################################

In [ ]:
acc_veh_df = accidents_df.join(vehicles_df, on="Accident_Index", how="inner")
casualties_df = casualties_df.withColumnRenamed("Vehicle_Reference", "Vehicle_Reference_Casualty")
full_df = acc_veh_df.join(casualties_df, on="Accident_Index", how="inner")

In [ ]:
full_df.show(5)

+--------------+---------------------+----------------------+---------+---------+------------+-----------------+------------------+--------------------+----------+-----------+-----+--------------------------+-------------------------+--------------+---------------+---------+-----------+---------------+----------------+--------------+---------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-------------------------+-----------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+-------------------------+---------------------------+--------------------------+-------------------+----------------------------+-------------------------+-------------+-------------+------------------+--------------------+-----

In [ ]:
len(full_df.columns)


67

In [ ]:
#############################################################################################################################
# Integrácia dát -  Sampling – vytvorenie vzorky z datasetu (veľkosti napr. 10%) pri zachovaní rozloženia cieľového atribútu.
#############################################################################################################################

In [ ]:
full_df.groupBy("Accident_Severity").count().orderBy("Accident_Severity").show()

+-----------------+-------+
|Accident_Severity|  count|
+-----------------+-------+
|                1|  83607|
|                2| 596571|
|                3|3607415|
+-----------------+-------+



In [ ]:
full_df.schema["Accident_Severity"].dataType

StringType()

In [ ]:
from pyspark.sql import functions as F

# proporcie (zachovanie rozlozenia)
class_distribution = full_df.groupBy("Accident_Severity").count().withColumn("fraction", F.col("count") / full_df.count())

In [ ]:
class_distribution.show(5)

+-----------------+-------+--------------------+
|Accident_Severity|  count|            fraction|
+-----------------+-------+--------------------+
|                3|3607415|  0.8413613418997559|
|                1|  83607|0.019499751958733023|
|                2| 596571|  0.1391389061415111|
+-----------------+-------+--------------------+



In [ ]:
#10% every class
fractions = {row["Accident_Severity"]: 0.1 for row in class_distribution.collect()}

In [ ]:
stratified_sample = full_df.sampleBy("Accident_Severity", fractions, seed=42)

In [ ]:
stratified_sample.groupBy("Accident_Severity").count().orderBy("Accident_Severity").show()

+-----------------+------+
|Accident_Severity| count|
+-----------------+------+
|                1|  8298|
|                2| 59781|
|                3|360153|
+-----------------+------+



In [ ]:
import os
os.makedirs("/content/drive/MyDrive/exports", exist_ok=True)

stratified_sample.toPandas().to_csv(
    "/content/drive/MyDrive/exports/stratified_sample.csv",
    index=False
)

In [ ]:
#############################################################################################################################
# Integrácia dát -  Rozdelenie datasetu na trénovaciu a testovaciu množinu (napr. v pomere 60/40).
#############################################################################################################################

In [ ]:
missing_count = stratified_sample.filter(F.col("Accident_Severity").isNull()).count()

In [ ]:
print(missing_count)

0


In [ ]:
# unique class cp
classes = [row["Accident_Severity"] for row in stratified_sample.select("Accident_Severity").distinct().collect()]

In [ ]:
print(classes)

['3', '1', '2']


In [ ]:
# zachovanie rozlozenia pre test a train
train_dfs = []
test_dfs = []

for cls in classes:
    # filter -> class
    cls_df = stratified_sample.filter(F.col("Accident_Severity") == cls)

    # class -> 60/40
    cls_train, cls_test = cls_df.randomSplit([0.6, 0.4], seed=42)

    train_dfs.append(cls_train)
    test_dfs.append(cls_test)

from functools import reduce
train = reduce(lambda a, b: a.union(b), train_dfs) #1.union(2).union(3)
test = reduce(lambda a, b: a.union(b), test_dfs)

In [ ]:
train.groupBy("Accident_Severity").count().orderBy("Accident_Severity").show()

+-----------------+------+
|Accident_Severity| count|
+-----------------+------+
|                1|  4926|
|                2| 36045|
|                3|216889|
+-----------------+------+



In [ ]:
test.groupBy("Accident_Severity").count().orderBy("Accident_Severity").show()

+-----------------+------+
|Accident_Severity| count|
+-----------------+------+
|                1|  3219|
|                2| 23461|
|                3|143595|
+-----------------+------+



In [ ]:
#############################################################################################################################
# Predspracovanie - Spracovanie chýbajúcich hodnôt (napr. ich nahradenie priemermi, atď.)
#############################################################################################################################

In [ ]:
train.printSchema()

root
 |-- Accident_Index: string (nullable = true)
 |-- Location_Easting_OSGR: string (nullable = true)
 |-- Location_Northing_OSGR: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Police_Force: string (nullable = true)
 |-- Accident_Severity: string (nullable = true)
 |-- Number_of_Vehicles: string (nullable = true)
 |-- Number_of_Casualties: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Day_of_Week: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Local_Authority_(District): string (nullable = true)
 |-- Local_Authority_(Highway): string (nullable = true)
 |-- 1st_Road_Class: string (nullable = true)
 |-- 1st_Road_Number: string (nullable = true)
 |-- Road_Type: string (nullable = true)
 |-- Speed_limit: string (nullable = true)
 |-- Junction_Detail: string (nullable = true)
 |-- Junction_Control: string (nullable = true)
 |-- 2nd_Road_Class: string (nullable = true)
 |-- 2nd_Road_Numb

In [ ]:
# delete unfunctional atr
columns_to_drop = [
    "Accident_Index",
    "Location_Easting_OSGR",
    "Location_Northing_OSGR",
    "Longitude",
    "Latitude",
    "Date",
    "Day_of_Week",
    "Time",
    "Local_Authority_(Highway)",
    "1st_Road_Number",
    "2nd_Road_Number",
    "LSOA_of_Accident_Location"
]

train = train.drop(*columns_to_drop)
test = test.drop(*columns_to_drop)

In [ ]:
len(train.columns)
len(test.columns)

55

In [ ]:
train.printSchema()

root
 |-- Police_Force: string (nullable = true)
 |-- Accident_Severity: string (nullable = true)
 |-- Number_of_Vehicles: string (nullable = true)
 |-- Number_of_Casualties: string (nullable = true)
 |-- Local_Authority_(District): string (nullable = true)
 |-- 1st_Road_Class: string (nullable = true)
 |-- Road_Type: string (nullable = true)
 |-- Speed_limit: string (nullable = true)
 |-- Junction_Detail: string (nullable = true)
 |-- Junction_Control: string (nullable = true)
 |-- 2nd_Road_Class: string (nullable = true)
 |-- Pedestrian_Crossing-Human_Control: string (nullable = true)
 |-- Pedestrian_Crossing-Physical_Facilities: string (nullable = true)
 |-- Light_Conditions: string (nullable = true)
 |-- Weather_Conditions: string (nullable = true)
 |-- Road_Surface_Conditions: string (nullable = true)
 |-- Special_Conditions_at_Site: string (nullable = true)
 |-- Carriageway_Hazards: string (nullable = true)
 |-- Urban_or_Rural_Area: string (nullable = true)
 |-- Did_Police_Office

In [ ]:
from pyspark.sql.functions import col, sum, when, isnull
null_counts = train.select([sum(when(isnull(c), 1).otherwise(0)).alias(c) for c in train.columns])
null_counts.show()

+------------+-----------------+------------------+--------------------+--------------------------+--------------+---------+-----------+---------------+----------------+--------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-----------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+-------------------------+---------------------------+--------------------------+-------------------+----------------------------+-------------------------+-------------+-------------+------------------+--------------------+---------------+--------------+-----------------+---------------------+--------------------------+------------------+--------------+---------------+---------------+--------------------+-------------

In [ ]:
minus_one_counts = train.select([
    sum(when(col(c) == "-1", 1).otherwise(0)).alias(c)
    for c in train.columns
])
minus_one_counts.show()

+------------+-----------------+------------------+--------------------+--------------------------+--------------+---------+-----------+---------------+----------------+--------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-----------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+-------------------------+---------------------------+--------------------------+-------------------+----------------------------+-------------------------+-------------+-------------+------------------+--------------------+---------------+--------------+-----------------+---------------------+--------------------------+------------------+--------------+---------------+---------------+--------------------+-------------

In [ ]:
#Junction_Detail: 3
#Junction_Control: 92515
#2nd_Road_Class: 107434
#Pedestrian_Crossing-Human_Control: 3
#Pedestrian_Crossing-Physical_Facilitie: 8
#Weather_Conditions: 25
#Road_Surface_Conditions: 293
#Special_Conditions_at_Site: 1
#Carriageway_Hazards: 3
#Did_Police_Officer_Attend_Scene_of_Accident: 34
#Vehicle_Type: 33
#Towing_and_Articulation: 13
#Vehicle_Manoeuvre: 82
#Vehicle_Location-Restricted_Lane: 3
#Junction_Location: 835
#Skidding_and_Overturning: 10
#Hit_Object_in_Carriageway: 6
#Vehicle_Leaving_Carriageway: 11
#Hit_Object_off_Carriageway: 3
#1st_Point_of_Impact: 151
#Was_Vehicle_Left_Hand_Drive?: 2006
#Journey_Purpose_of_Driver: 3816
#Age_of_Driver: 24218
#Age_Band_of_Driver: 24218
#Engine_Capacity_(CC): 61590
#Propulsion_Code: 60027
#Age_of_Vehicle: 69208
#Driver_IMD_Decile: 70087
#Driver_Home_Area_Type: 47354
#Sex_of_Casualty: 79
#Age_of_Casualty: 5107
#Age_Band_of_Casualty: 5107
#Pedestrian_Location: 1
#Pedestrian_Movement: 1
#Car_Passenger: 57
#Bus_or_Coach_Passenger: 1
#Pedestrian_Road_Maintenance_Worker: 167116
#Casualty_Home_Area_Type: 36273

In [ ]:
minus_one_counts = test.select([
    sum(when(col(c) == "-1", 1).otherwise(0)).alias(c)
    for c in test.columns
])
minus_one_counts.show()

+------------+-----------------+------------------+--------------------+--------------------------+--------------+---------+-----------+---------------+----------------+--------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-----------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+-------------------------+---------------------------+--------------------------+-------------------+----------------------------+-------------------------+-------------+-------------+------------------+--------------------+---------------+--------------+-----------------+---------------------+--------------------------+------------------+--------------+---------------+---------------+--------------------+-------------

In [ ]:
#Junction_Detail: 1
#Junction_Control: 60857
#2nd_Road_Class: 70823
#Pedestrian_Crossing-Human_Control: 1
#Pedestrian_Crossing-Physical_Facilities: 3
#Weather_Conditions: 16
#Road_Surface_Conditions: 221
#Special_Conditions_at_Site: 1
#Carriageway_Hazards: 4
#Did_Police_Officer_Attend_Scene_of_Accident: 29
#Vehicle_Type:13
#Towing_and_Articulation: 7
#Vehicle_Manoeuvre: 58
#Vehicle_Location-Restricted_Lane: 4
#Junction_Location: 573
#Skidding_and_Overturning: 7
#Hit_Object_in_Carriageway: 2
#Vehicle_Leaving_Carriageway: 6
#Hit_Object_off_Carriageway: 1
#1st_Point_of_Impact: 89
#Was_Vehicle_Left_Hand_Drive?: 1410
#Journey_Purpose_of_Driver: 2599
#Sex_of_Driver: 2
#Age_of_Driver: 15919
#Age_Band_of_Driver: 15919
#Engine_Capacity_(CC): 40863
#Propulsion_Code: 39859
#Age_of_Vehicle: 45931
#Driver_IMD_Decile: 46255
#Driver_Home_Area_Type: 31158
#Sex_of_Casualty: 48
#Age_of_Casualty: 3485
#Age_Band_of_Casualty: 3485
#Car_Passenger: 36
#Bus_or_Coach_Passenger: 4
#Pedestrian_Road_Maintenance_Worker: 110398
#Casualty_Home_Area_Type: 23961

In [ ]:
###################################################
# drop colums: >27% "-1"
###################################################

In [ ]:
columns_to_drop = [
    "Pedestrian_Road_Maintenance_Worker",
    "2nd_Road_Class",
    "Driver_IMD_Decile"
]

In [ ]:
train = train.drop(*columns_to_drop)
test = test.drop(*columns_to_drop)

In [ ]:
len(test.columns)

52

In [ ]:
train.show(10)

+------------+-----------------+------------------+--------------------+--------------------------+--------------+---------+-----------+---------------+----------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-----------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+-------------------------+---------------------------+--------------------------+-------------------+----------------------------+-------------------------+-------------+-------------+------------------+--------------------+---------------+--------------+---------------------+--------------------------+------------------+--------------+---------------+---------------+--------------------+-----------------+-------------------+--------

In [ ]:
train.printSchema()

root
 |-- Police_Force: string (nullable = true)
 |-- Accident_Severity: string (nullable = true)
 |-- Number_of_Vehicles: string (nullable = true)
 |-- Number_of_Casualties: string (nullable = true)
 |-- Local_Authority_(District): string (nullable = true)
 |-- 1st_Road_Class: string (nullable = true)
 |-- Road_Type: string (nullable = true)
 |-- Speed_limit: string (nullable = true)
 |-- Junction_Detail: string (nullable = true)
 |-- Junction_Control: string (nullable = true)
 |-- Pedestrian_Crossing-Human_Control: string (nullable = true)
 |-- Pedestrian_Crossing-Physical_Facilities: string (nullable = true)
 |-- Light_Conditions: string (nullable = true)
 |-- Weather_Conditions: string (nullable = true)
 |-- Road_Surface_Conditions: string (nullable = true)
 |-- Special_Conditions_at_Site: string (nullable = true)
 |-- Carriageway_Hazards: string (nullable = true)
 |-- Urban_or_Rural_Area: string (nullable = true)
 |-- Did_Police_Officer_Attend_Scene_of_Accident: string (nullable =

In [ ]:
#################################################
# atribute type convert
#################################################

In [ ]:
numeric_columns = [
    "Number_of_Vehicles",
    "Number_of_Casualties",
    "Speed_limit",
    "Age_of_Driver",
    "Age_Band_of_Driver",
    "Engine_Capacity_(CC)",
    "Age_of_Vehicle",
    "Age_of_Casualty",
    "Age_Band_of_Casualty"
]


In [ ]:
categorical_columns = [
    "Police_Force",
    "Local_Authority_(District)",
    "1st_Road_Class",
    "Junction_Detail",
    "Junction_Control",
    "Pedestrian_Crossing-Human_Control",
    "Pedestrian_Crossing-Physical_Facilities",
    "Light_Conditions",
    "Weather_Conditions",
    "Road_Surface_Conditions",
    "Special_Conditions_at_Site",
    "Carriageway_Hazards",
    "Urban_or_Rural_Area",
    "Did_Police_Officer_Attend_Scene_of_Accident",
    "Vehicle_Reference",
    "Vehicle_Type",
    "Towing_and_Articulation",
    "Vehicle_Manoeuvre",
    "Vehicle_Location-Restricted_Lane",
    "Junction_Location",
    "Skidding_and_Overturning",
    "Hit_Object_in_Carriageway",
    "Vehicle_Leaving_Carriageway",
    "Hit_Object_off_Carriageway",
    "1st_Point_of_Impact",
    "Was_Vehicle_Left_Hand_Drive?",
    "Journey_Purpose_of_Driver",
    "Sex_of_Driver",
    "Propulsion_Code",
    "Driver_Home_Area_Type",
    "Vehicle_Reference_Casualty",
    "Casualty_Reference",
    "Casualty_Class",
    "Sex_of_Casualty",
    "Casualty_Severity",
    "Pedestrian_Location",
    "Pedestrian_Movement",
    "Car_Passenger",
    "Bus_or_Coach_Passenger",
    "Casualty_Type",
    "Casualty_Home_Area_Type"
]


In [ ]:
######################################
# string - > num
######################################

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

numeric_types = {
    "Number_of_Vehicles": IntegerType(),
    "Number_of_Casualties": IntegerType(),
    "Speed_limit": IntegerType(),
    "Age_of_Driver": IntegerType(),
    "Age_Band_of_Driver": IntegerType(),
    "Engine_Capacity_(CC)": IntegerType(),
    "Age_of_Vehicle": IntegerType(),
    "Age_of_Casualty": IntegerType(),
    "Age_Band_of_Casualty": IntegerType()
}

def convert_columns(df, columns_dict):
    for column, dtype in columns_dict.items():
        df = df.withColumn(column, col(column).cast(dtype))
    return df


In [ ]:
train = convert_columns(train, numeric_types)
test = convert_columns(test, numeric_types)

In [ ]:
test.select(numeric_columns).printSchema()

root
 |-- Number_of_Vehicles: integer (nullable = true)
 |-- Number_of_Casualties: integer (nullable = true)
 |-- Speed_limit: integer (nullable = true)
 |-- Age_of_Driver: integer (nullable = true)
 |-- Age_Band_of_Driver: integer (nullable = true)
 |-- Engine_Capacity_(CC): integer (nullable = true)
 |-- Age_of_Vehicle: integer (nullable = true)
 |-- Age_of_Casualty: integer (nullable = true)
 |-- Age_Band_of_Casualty: integer (nullable = true)



In [ ]:
###################################################
# num features: -1 to median
###################################################

In [ ]:
columns_to_fix = [
    "Age_of_Vehicle",
    "`Engine_Capacity_(CC)`",
    "Age_Band_of_Driver",
    "Age_of_Driver"
]

In [ ]:
from pyspark.sql import functions as F

for col_name in columns_to_fix:
    median = train.where(F.col(col_name) != -1) \
                  .approxQuantile(col_name, [0.5], 0.01)[0]

In [ ]:
train = train.withColumn(
       col_name,
       F.when(F.col(col_name) == -1, F.lit(median)).otherwise(F.col(col_name))
)

In [ ]:
test = test.withColumn(
       col_name,
       F.when(F.col(col_name) == -1, F.lit(median)).otherwise(F.col(col_name))
)

In [ ]:
minus_one_counts = train.select([
    sum(when(col(c) == "-1", 1).otherwise(0)).alias(c)
    for c in train.columns
])
minus_one_counts.show()

+------------+-----------------+------------------+--------------------+--------------------------+--------------+---------+-----------+---------------+----------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-----------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+-------------------------+---------------------------+--------------------------+-------------------+----------------------------+-------------------------+-------------+-------------+------------------+--------------------+---------------+--------------+---------------------+--------------------------+------------------+--------------+---------------+---------------+--------------------+-----------------+-------------------+--------

In [ ]:
median_age_of_vehicle = train.where(F.col("Age_of_Vehicle") != -1) \
                             .approxQuantile("Age_of_Vehicle", [0.5], 0.01)[0]

In [ ]:
train = train.withColumn(
    "Age_of_Vehicle",
    F.when(F.col("Age_of_Vehicle") == -1, median_age_of_vehicle)
     .otherwise(F.col("Age_of_Vehicle"))
)

In [ ]:
count_minus_one = train.filter(F.col("Age_of_Vehicle") == -1).count()

In [ ]:
count_minus_one

0

In [ ]:
train.printSchema()

root
 |-- Police_Force: string (nullable = true)
 |-- Accident_Severity: string (nullable = true)
 |-- Number_of_Vehicles: integer (nullable = true)
 |-- Number_of_Casualties: integer (nullable = true)
 |-- Local_Authority_(District): string (nullable = true)
 |-- 1st_Road_Class: string (nullable = true)
 |-- Road_Type: string (nullable = true)
 |-- Speed_limit: integer (nullable = true)
 |-- Junction_Detail: string (nullable = true)
 |-- Junction_Control: string (nullable = true)
 |-- Pedestrian_Crossing-Human_Control: string (nullable = true)
 |-- Pedestrian_Crossing-Physical_Facilities: string (nullable = true)
 |-- Light_Conditions: string (nullable = true)
 |-- Weather_Conditions: string (nullable = true)
 |-- Road_Surface_Conditions: string (nullable = true)
 |-- Special_Conditions_at_Site: string (nullable = true)
 |-- Carriageway_Hazards: string (nullable = true)
 |-- Urban_or_Rural_Area: string (nullable = true)
 |-- Did_Police_Officer_Attend_Scene_of_Accident: string (nullabl

In [ ]:
count_minus_one_1 = train.filter(F.col("Age_of_Vehicle") == "-1").count()

In [ ]:
count_minus_one_1

0

In [ ]:
test = test.withColumn(
    "Age_of_Vehicle",
    F.when(F.col("Age_of_Vehicle") == -1, median_age_of_vehicle)
     .otherwise(F.col("Age_of_Vehicle"))
)

In [ ]:
median_Engine_Capacity = train.where(F.col("`Engine_Capacity_(CC)`") != -1) \
                             .approxQuantile("`Engine_Capacity_(CC)`", [0.5], 0.01)[0]

In [ ]:
median_Engine_Capacity

1598.0

In [ ]:
train = train.withColumn(
    "`Engine_Capacity_(CC)`",
    F.when(F.col("`Engine_Capacity_(CC)`") == -1, median_Engine_Capacity)
     .otherwise(F.col("`Engine_Capacity_(CC)`"))
)
test = test.withColumn(
    "`Engine_Capacity_(CC)`",
    F.when(F.col("`Engine_Capacity_(CC)`") == -1, median_Engine_Capacity)
     .otherwise(F.col("`Engine_Capacity_(CC)`"))
)

In [ ]:
median_Age_Band_of_Driver = train.where(F.col("Age_Band_of_Driver") != -1) \
                             .approxQuantile("Age_Band_of_Driver", [0.5], 0.01)[0]

In [ ]:
median_Age_Band_of_Driver

7.0

In [ ]:
train = train.withColumn(
    "Age_Band_of_Driver",
    F.when(F.col("Age_Band_of_Driver") == -1, median_Age_Band_of_Driver)
     .otherwise(F.col("Age_Band_of_Driver"))
)
test = test.withColumn(
    "Age_Band_of_Driver",
    F.when(F.col("Age_Band_of_Driver") == -1, median_Age_Band_of_Driver)
     .otherwise(F.col("Age_Band_of_Driver"))
)

In [ ]:
median_Age_of_Driver = train.where(F.col("Age_of_Driver") != -1) \
                             .approxQuantile("Age_of_Driver", [0.5], 0.01)[0]

In [ ]:
median_Age_of_Driver

36.0

In [ ]:
train = train.withColumn(
    "Age_of_Driver",
    F.when(F.col("Age_of_Driver") == -1, median_age_of_vehicle)
     .otherwise(F.col("Age_of_Driver"))
)
test = test.withColumn(
    "Age_of_Driver",
    F.when(F.col("Age_of_Driver") == -1, median_age_of_vehicle)
     .otherwise(F.col("Age_of_Driver"))
)

In [ ]:
###################################################
# categoracil features: -1 to "Missing"
###################################################

In [ ]:
columns_to_fix_ctg = [
    "Junction_Control",
    "Propulsion_Code",
    "Driver_Home_Area_Type",
    "Casualty_Home_Area_Type"
]

In [ ]:
for col_name in columns_to_fix_ctg:
    train = train.withColumn(col_name, F.when(F.col(col_name) == -1, "Missing").otherwise(F.col(col_name)))
    test = test.withColumn(col_name, F.when(F.col(col_name) == -1, "Missing").otherwise(F.col(col_name)))


In [ ]:
minus_one_counts = train.select([
    sum(when(col(c) == "-1", 1).otherwise(0)).alias(c)
    for c in train.columns
])
minus_one_counts.show()

+------------+-----------------+------------------+--------------------+--------------------------+--------------+---------+-----------+---------------+----------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-----------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+-------------------------+---------------------------+--------------------------+-------------------+----------------------------+-------------------------+-------------+-------------+------------------+--------------------+---------------+--------------+---------------------+--------------------------+------------------+--------------+---------------+---------------+--------------------+-----------------+-------------------+--------

In [ ]:
##########################################################
# delete rows with <5% missing + eng cap atr del
##########################################################

In [ ]:
columns_to_drop = [
    "`Engine_Capacity_(CC)`",
    "Engine_Capacity_(CC)"
]

train = train.drop(*columns_to_drop)
test = test.drop(*columns_to_drop)

In [ ]:
columns_to_clean = [
    "Age_of_Casualty",
    "Age_Band_of_Casualty",
    "Journey_Purpose_of_Driver",
    "Was_Vehicle_Left_Hand_Drive?",
    "Junction_Location",
    "Road_Surface_Conditions",
    "1st_Point_of_Impact",
    "Vehicle_Manoeuvre",
    "Sex_of_Casualty",
    "Car_Passenger",
    "Did_Police_Officer_Attend_Scene_of_Accident",
    "Weather_Conditions",
    "Vehicle_Type",
    "Towing_and_Articulation",
    "Skidding_and_Overturning",
    "Vehicle_Leaving_Carriageway",
    "Vehicle_Location-Restricted_Lane",
    "Bus_or_Coach_Passenger",
    "Carriageway_Hazards",
    "Pedestrian_Crossing-Physical_Facilities",
    "Sex_of_Driver",
    "Hit_Object_in_Carriageway",
    "Junction_Detail",
    "Special_Conditions_at_Site",
    "Pedestrian_Crossing-Human_Control",
    "Hit_Object_off_Carriageway"
]

for col_name in columns_to_clean:
    train = train.filter(F.col(col_name) != -1)
    test = test.filter(F.col(col_name) != -1)


In [ ]:
minus_one_counts = train.select([
    sum(when(col(c) == -1, 1).otherwise(0)).alias(c)
    for c in train.columns
])
minus_one_counts.show()

+------------+-----------------+------------------+--------------------+--------------------------+--------------+---------+-----------+---------------+----------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-----------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+-------------------------+---------------------------+--------------------------+-------------------+----------------------------+-------------------------+-------------+-------------+------------------+---------------+--------------+---------------------+--------------------------+------------------+--------------+---------------+---------------+--------------------+-----------------+-------------------+-------------------+---------

In [ ]:
columns_to_clean = [
    "Pedestrian_Location",
    "Pedestrian_Movement"
]

for col_name in columns_to_clean:
    train = train.filter(F.col(col_name) != -1)
    test = test.filter(F.col(col_name) != -1)


In [ ]:
minus_one_counts = train.select([
    sum(when(col(c) == -1, 1).otherwise(0)).alias(c)
    for c in train.columns
])
minus_one_counts.show()

+------------+-----------------+------------------+--------------------+--------------------------+--------------+---------+-----------+---------------+----------------+---------------------------------+---------------------------------------+----------------+------------------+-----------------------+--------------------------+-------------------+-------------------+-------------------------------------------+-----------------+------------+-----------------------+-----------------+--------------------------------+-----------------+------------------------+-------------------------+---------------------------+--------------------------+-------------------+----------------------------+-------------------------+-------------+-------------+------------------+---------------+--------------+---------------------+--------------------------+------------------+--------------+---------------+---------------+--------------------+-----------------+-------------------+-------------------+---------

In [ ]:
train.write.mode("append").parquet("./drive/MyDrive/dataset/train_without_missing.parquet")
test.write.mode("append").parquet("./drive/MyDrive/dataset/test_without_missing.parquet")